In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
import matplotlib.lines as mlines
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM  # for novelty detection
from sklearn.inspection import DecisionBoundaryDisplay

# 1️⃣ Load the Iris dataset
iris = load_iris()
X = iris.data[:, :2]  # use first two features: sepal length & width
y = iris.target

# 2️⃣ Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3️⃣ Train a model (One-Class SVM for novelty detection)
clf = OneClassSVM(kernel="rbf", gamma=0.1, nu=0.1)
clf.fit(X_train)

# 4️⃣ Generate outliers (for visualization)
rng = np.random.RandomState(42)
X_outliers = rng.uniform(low=-1, high=8, size=(40, 2))

# 5️⃣ Predict errors
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)
y_pred_outliers = clf.predict(X_outliers)

n_error_train = y_pred_train[y_pred_train == -1].size
n_error_test = y_pred_test[y_pred_test == -1].size
n_error_outliers = y_pred_outliers[y_pred_outliers == 1].size

# 6️⃣ Decision boundary visualization
_, ax = plt.subplots(figsize=(8, 6))

xx, yy = np.meshgrid(np.linspace(-1, 8, 500), np.linspace(-1, 8, 500))
X_grid = np.c_[xx.ravel(), yy.ravel()]

DecisionBoundaryDisplay.from_estimator(
    clf,
    X_grid,
    response_method="decision_function",
    plot_method="contourf",
    ax=ax,
    cmap="PuBu",
)

DecisionBoundaryDisplay.from_estimator(
    clf,
    X_grid,
    response_method="decision_function",
    plot_method="contour",
    ax=ax,
    levels=[0],
    colors="darkred",
    linewidths=2,
)

s = 40
b1 = ax.scatter(X_train[:, 0], X_train[:, 1], c="white", s=s, edgecolors="k")
b2 = ax.scatter(X_test[:, 0], X_test[:, 1], c="blueviolet", s=s, edgecolors="k")
c = ax.scatter(X_outliers[:, 0], X_outliers[:, 1], c="gold", s=s, edgecolors="k")

plt.legend(
    [mlines.Line2D([], [], color="darkred"), b1, b2, c],
    [
        "learned frontier",
        "training observations",
        "new regular observations",
        "new abnormal observations",
    ],
    loc="upper left",
    prop=matplotlib.font_manager.FontProperties(size=10),
)

ax.set(
    xlabel=(f"Train errors: {n_error_train}, Test errors: {n_error_test}, Outliers: {n_error_outliers}"),
    ylabel="Sepal width (cm)",
    title="🌸 Iris Dataset - Novelty Detection (SVM)",
    xlim=(-1, 8),
    ylim=(-1, 8),
)

plt.show()
